In [85]:
# Download important libraries
%matplotlib inline
import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
np.random.seed(42)
tf.random.set_seed(42)
import json
import os
from os import path
import tarfile
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import array_contains
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher
from pyspark.sql.functions import udf
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import regexp_replace, col
from functools import reduce
pipeline=None
import time

In [20]:
spark = sparknlp.start(gpu=True) 

In [21]:
spark

In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Spark\\spark-3.1.1-bin-hadoop2.7'

In [19]:
spark.stop()

In [3]:
# spark = SparkSession.builder \
#     .appName("Spark NLP")\
#     .master("local[*]")\
#     .config("spark.driver.memory","16G")\
#     .config("spark.driver.maxResultSize", "0") \
#     .config("spark.kryoserializer.buffer.max", "2000M")\
#     .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.0").getOrCreate()
# spark

In [87]:
def clean_df(df):
    df = df[df.caption != '']
    
    return df.reset_index(drop=True)

In [88]:
import warnings
warnings.filterwarnings('ignore')

In [89]:
st_time = time.time()
rev_dpath = '../data/outputs/reviews/'
        
rev_files = [rev_dpath+ f for f in os.listdir(rev_dpath)][:3000]
df_rev = pd.DataFrame()
for file in rev_files:
    
    df = pd.read_json(file)
    df_clean = clean_df(df)
    df_clean['place_id'] = file.split('_',1)[1]
    df_clean['place_id'] = df_clean['place_id'].str.replace('.json','')
    df_rev = pd.concat((df_rev, df_clean), ignore_index=True)
df_rev.to_parquet('../data/spark/reviews/test.parq')

print(time.time()-st_time)

504.74505138397217


In [15]:
# # rev_dpath_jsonl = '../data/spark/reviews/jsonl/'
# # os.makedirs(rev_dpath_jsonl, exist_ok=True)
# rev_files_jsonl = [rev_dpath_jsonl+ f for f in os.listdir(rev_dpath_jsonl)]

rev_dpath = '../data/outputs/reviews/'
        
rev_files = [rev_dpath+ f for f in os.listdir(rev_dpath)][:1000]
df_rev = pd.DataFrame()
for file in rev_files:
    
    df = pd.read_json(file)

    df_rev = pd.concat((df_rev, df), ignore_index=True)
    
df_rev.to_parquet('../data/spark/reviews/test.parq')


In [132]:
from pyspark.sql.functions import regexp_replace, col

trainDataset = spark.read.option("header", True).parquet('../data/spark/reviews/test.parq').select(col("caption"),col("rating"))
trainDataset = trainDataset.withColumn("caption", regexp_replace(("caption"), "(Translated by Google)", ""))
df = trainDataset.repartition(1)
trainDataset.show(10)
df_filtered=df.filter(df.caption !='')
df_filtered = df_filtered.withColumn("rating", col("rating").cast("double"))

+--------------------+------+
|             caption|rating|
+--------------------+------+
|One of my new fav...|     5|
|I got the Peanut ...|     5|
|Love this place! ...|     5|
|Everything is Ama...|     5|
|The bacon, egg, a...|     5|
|We really like th...|     5|
|Quick service, ta...|     5|
|Best BBQ...even b...|     5|
|So nice the break...|     5|
|Best place for br...|     5|
+--------------------+------+
only showing top 10 rows



In [133]:

df_filtered.groupBy("rating") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+------+------+
|rating| count|
+------+------+
|   5.0|232193|
|   4.0| 65204|
|   1.0| 30619|
|   3.0| 26711|
|   2.0| 14869|
+------+------+



In [135]:
train_df, test_df = df_filtered.randomSplit([0.8,0.2],42)
train_df.show(10)

+--------------------+------+
|             caption|rating|
+--------------------+------+
|         ! AMAZING!!|   5.0|
|! Best BBQ in the...|   5.0|
|"#MUCH_LOVE_AND_M...|   4.0|
|"A 3rd Place wher...|   5.0|
|"A must have when...|   5.0|
|"Amazing food, gr...|   5.0|
|"Best meal,  expe...|   5.0|
|"Best place to ea...|   5.0|
|"Best wings in th...|   5.0|
|"Big easy" noodle...|   5.0|
+--------------------+------+
only showing top 10 rows



In [143]:
import pyspark.sql.types

document = DocumentAssembler()\
    .setInputCol("caption")\
    .setOutputCol("document")

# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence
# downloading pretrained embeddings

embeddings = UniversalSentenceEncoder.pretrained('tfhub_use', lang="en") \
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("rating")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)
use_clf_pipeline = Pipeline(
    stages = [
        document,
        embeddings,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


IllegalArgumentException: requirement failed: Was not found appropriate resource to download for request: ResourceRequest(tfhub_use,Some(en),public/models,3.0.0,3.1.1) with downloader: com.johnsnowlabs.nlp.pretrained.S3ResourceDownloader@6956083c

In [142]:
use_pipelineModel = use_clf_pipeline.fit(df_filtered)


IllegalArgumentException: requirement failed: Wrong or missing inputCols annotators in SentenceEmbeddings_2b0176c4facf.

Current inputCols: document. Dataset's columns:
(column_name=caption,is_nlp_annotator=false)
(column_name=rating,is_nlp_annotator=false)
(column_name=document,is_nlp_annotator=true,type=document).
Make sure such annotators exist in your pipeline, with the right output names and that they have following annotator types: document, word_embeddings

In [116]:
st_time = time.time()
rev_dpath = '../data/outputs/reviews/'
        
rev_files = [rev_dpath+ f for f in os.listdir(rev_dpath)]
df_rev = pd.DataFrame()
for file in rev_files:
    
    df = pd.read_json(file)

    df_rev = pd.concat((df_rev, df), ignore_index=True)
print(time.time()-st_time)

PermissionError: [Errno 13] Permission denied: '../data/outputs/reviews/jsonl'

In [97]:
rev_files[0]

'../data/outputs/reviews/10810402004_ChIJ2Wl4t3vzjIgR0jrQX9_UoWw.json'

In [99]:
with open(rev_files[0],'r') as inp_file:
    x= json.load(inp_file)
x

[{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN5LWZtWmZnEAE',
  'caption': 'One of my new favorite places 😋',
  'relative_date': 'a month ago',
  'retrieval_date': '2021-03-23',
  'rating': 5.0,
  'username': 'Monica Crist',
  'n_review_user': 20,
  'n_photo_user': 0,
  'url_user': 'https://www.google.com/maps/contrib/117300709159619750156/reviews?hl=en-US',
  'census_block_group': 10810402004},
 {'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUQ4dTVxcWFREAE',
  'caption': 'I got the Peanut Butter Cup Shake and it was really good! Cant wait to try more flavors!',
  'relative_date': '7 months ago',
  'retrieval_date': '2021-03-23',
  'rating': 5.0,
  'username': 'Paulina White',
  'n_review_user': 0,
  'n_photo_user': 0,
  'url_user': 'https://www.google.com/maps/contrib/106425158284488465525/reviews?hl=en-US',
  'census_block_group': 10810402004},
 {'id_review': 'ChdDSUhNMG9nS0VJQ0FnSURjNXVDM25nRRAB',
  'caption': 'Love this place! I recommend the cucumber watermelon loaded tea and the cake batter shake! 

In [ ]:
st_time = time.time()
rev_dpath_jsonl = '../data/spark/reviews/jsonl/'
os.makedirs(rev_dpath_jsonl, exist_ok=True)
rev_files_jsonl = [rev_dpath_jsonl+ f for f in os.listdir(rev_dpath_jsonl)]
for file in rev_files:
    JSON_file = json.loads()
with open('output.jsonl', 'w') as outfile:
    with open
    for entry in JSON_file:
        json.dump(entry, outfile)
        outfile.write('\n')

In [115]:
st_time = time.time()
rev_dpath_jsonl = '../data/spark/reviews/jsonl/'
os.makedirs(rev_dpath_jsonl, exist_ok=True)
rev_files_jsonl = [rev_dpath_jsonl+ f for f in os.listdir(rev_dpath_jsonl)]
dataframes = map(lambda r: spark.read.json(r), rev_files_jsonl)
union = reduce(lambda df1, df2: df1.unionAll(df2), dataframes)
df = union.collect()
print(time.time()-st_time)

2404.066113471985


In [86]:
file = rev_dpath_jsonl = '../data/outputs/reviews/jsonl/10730027002_ChIJNe0mPMkbiYgR6oApk2Pt7F8.json'

spark.read.json(file).head(2)[0]

Row(caption='', census_block_group=10730027002, id_review='ChZDSUhNMG9nS0VJQ0FnSURJME52MFhBEAE', n_photo_user=0, n_review_user=0, rating=4.0, relative_date='2 years ago', retrieval_date='2021-03-23', url_user='https://www.google.com/maps/contrib/116059736436598578008/reviews?hl=en-US', username='MIGUEL BOSCAN')

In [17]:
df

DataFrame[_corrupt_record: string]